In [1]:
import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr

from sklearn.linear_model import LinearRegression
from tqdm import tqdm

## Get Stock List

In [2]:
path = './'
list_name = 'stock_list.csv'
sample_name = 'sample_submission.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


## Get Data & Modeling

In [3]:
start_date = '20210104'
end_date = '20211105'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 44
HOW MANY "Business_days" : (220, 1)


,Date
0,2021-01-04
1,2021-01-05
2,2021-01-06
3,2021-01-07
4,2021-01-08


## Baseline 모델의 구성 소개 ( Sample )

- X : (월 ~ 금) * 43주간
- y : (다음주 월 ~ 금) * 43주간
    - y_0 : 다음주 월요일
    - y_1 : 다음주 화요일
    - y_2 : 다음주 수요일
    - y_3 : 다음주 목요일
    - y_4 : 다음주 금요일


- 이번주 월~금요일의 패턴을 학습해 다음주 월요일 ~ 금요일을 각각 예측하는 모델을 생성
    
- 이 과정을 모든 종목(370개)에 적용

In [4]:
sample_code = stock_list.loc[0,'종목코드']

sample = fdr.DataReader(sample_code, start = start_date, end = end_date)[['Close']].reset_index()
sample = pd.merge(Business_days, sample, how = 'outer')
sample['weekday'] = sample.Date.apply(lambda x : x.weekday())
sample['weeknum'] = sample.Date.apply(lambda x : x.strftime('%V'))
sample.Close = sample.Close.ffill()
sample = pd.pivot_table(data = sample, values = 'Close', columns = 'weekday', index = 'weeknum')
sample.head()

weekday,0,1,2,3,4
weeknum,,,,,
01,83000.0,83900.0,82200.0,82900.0,88800.0
02,91000.0,90600.0,89700.0,89700.0,88000.0
03,85000.0,87000.0,87200.0,88100.0,86800.0
04,89400.0,86700.0,85600.0,83700.0,82000.0
05,83000.0,84400.0,84600.0,82500.0,83500.0


In [5]:
model = LinearRegression()

In [20]:
x = sample.iloc[0:-2].to_numpy()
x.shape

(42, 5)

In [21]:
x

array([[83000., 83900., 82200., 82900., 88800.],
       [91000., 90600., 89700., 89700., 88000.],
       [85000., 87000., 87200., 88100., 86800.],
       [89400., 86700., 85600., 83700., 82000.],
       [83000., 84400., 84600., 82500., 83500.],
       [83000., 82700., 81600., 81600., 81600.],
       [84200., 84900., 83200., 82100., 82600.],
       [82200., 82000., 82000., 85300., 82500.],
       [82500., 83600., 84000., 82400., 82100.],
       [82000., 81400., 80900., 82000., 82800.],
       [81800., 82800., 82300., 82900., 81900.],
       [82000., 81800., 81000., 81200., 81500.],
       [81600., 82200., 81400., 82900., 84800.],
       [85400., 86000., 85600., 84700., 83600.],
       [83200., 84000., 84000., 84100., 83900.],
       [83300., 83900., 82600., 82400., 82800.],
       [83500., 82900., 82100., 81700., 81500.],
       [81700., 82600., 82600., 82300., 81900.],
       [83200., 81200., 80000., 78500., 80100.],
       [79600., 79600., 79600., 79500., 80100.],
       [79700., 7990

In [7]:
y = sample.iloc[1:-1].to_numpy()
y_0 = y[:,0]
y_1 = y[:,1]
y_2 = y[:,2]
y_3 = y[:,3]
y_4 = y[:,4]

y_values = [y_0, y_1, y_2, y_3, y_4]

In [8]:
x_public = sample.iloc[-2].to_numpy()

- 예측

In [9]:
predictions = []
for y_value in y_values :
    model.fit(x,y_value)
    prediction = model.predict(np.expand_dims(x_public,0))
    predictions.append(prediction[0])
predictions

[70206.67660106532,
 69631.42785252717,
 69062.32129096358,
 69258.21096883612,
 68846.00977524316]

- 실제 Public 값

In [10]:
sample.iloc[-1].values

array([69900., 71500., 70400., 70600., 70200.])

# 전체 모델링

In [11]:
sample_name = 'sample_submission.csv'
sample_submission = pd.read_csv(os.path.join(path,sample_name))

In [12]:
model = LinearRegression()
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    data = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    
    x = data.iloc[0:-2].to_numpy() # 2021년 1월 04일 ~ 2021년 10월 22일까지의 데이터로
    y = data.iloc[1:-1].to_numpy() # 2021년 1월 11일 ~ 2021년 10월 29일까지의 데이터를 학습한다.
    y_0 = y[:,0]
    y_1 = y[:,1]
    y_2 = y[:,2]
    y_3 = y[:,3]
    y_4 = y[:,4]

    y_values = [y_0, y_1, y_2, y_3, y_4]
    x_public = data.iloc[-2].to_numpy() # 2021년 11월 1일부터 11월 5일까지의 데이터를 예측할 것이다.
    
    predictions = []
    for y_value in y_values :
        model.fit(x,y_value)
        prediction = model.predict(np.expand_dims(x_public,0))
        predictions.append(prediction[0])
    sample_submission.loc[:,code] = predictions * 2
sample_submission.isna().sum().sum()

100%|██████████| 370/370 [00:33<00:00, 11.11it/s]


0

In [19]:
x

array([[31950., 32950., 35350., 38200., 38300.],
       [37850., 41200., 38600., 36800., 37550.],
       [43550., 44350., 52000., 67600., 63700.],
       [67700., 66000., 61600., 59000., 59500.],
       [59100., 61200., 63300., 68900., 67300.],
       [68200., 65900., 75100., 75100., 75100.],
       [78400., 74200., 70200., 73500., 72100.],
       [73500., 72000., 64900., 65700., 62100.],
       [62100., 60600., 62300., 62000., 61800.],
       [60000., 61000., 58100., 59400., 60900.],
       [61200., 61900., 59000., 63300., 61200.],
       [60000., 57700., 59900., 61500., 61600.],
       [65300., 63300., 61800., 60300., 60400.],
       [59800., 57800., 57800., 57000., 57100.],
       [55600., 56300., 57200., 56100., 54900.],
       [56600., 57200., 55600., 55200., 53800.],
       [54200., 53000., 51400., 51100., 51100.],
       [50000., 51000., 51000., 51300., 53200.],
       [52900., 50900., 50100., 47800., 48950.],
       [47750., 49900., 49900., 49300., 49750.],
       [50800., 5450

In [13]:
sample_submission.columns

Index(['Day', '000060', '000080', '000100', '000120', '000150', '000240',
       '000250', '000270', '000660',
       ...
       '330860', '336260', '336370', '347860', '348150', '348210', '352820',
       '357780', '363280', '950130'],
      dtype='object', length=371)

In [14]:
columns = list(sample_submission.columns[1:])

columns = ['Day'] + [str(x).zfill(6) for x in columns]

sample_submission.columns = columns

In [15]:
sample_submission.to_csv('BASELINE_Linear.csv',index=False)

In [16]:
sample_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,27919.530611,34687.673458,60773.779528,142621.815394,104901.698658,16669.447967,47219.595113,85236.833170,103490.352393,...,49749.405974,51984.322942,84384.021784,36846.592704,25721.026664,53328.350326,336697.743579,262257.538308,27176.085090,17382.219194
1,2021-11-02,28750.750484,35032.651375,60312.130021,143012.927861,107216.342323,17001.594758,46672.351191,85360.327648,102788.687368,...,48923.253693,51539.564130,84739.157977,35258.227509,25330.528060,53645.466661,335662.149461,264562.230652,27415.549191,17338.522537
2,2021-11-03,28858.095631,34995.888574,60241.470410,145626.792237,111192.733424,17665.577952,45757.683516,85665.326378,102943.419081,...,48834.428543,49341.153729,85450.806060,35320.479243,25623.131825,55136.121480,329167.718872,264623.119599,27466.446666,17286.577226
3,2021-11-04,28901.301911,34866.098057,59701.569734,145351.536595,109530.741544,17846.022961,46380.152350,85494.864447,99958.476851,...,48954.935640,48626.256108,84823.889868,34343.016848,25878.115140,55964.135511,329482.609718,261821.140588,27594.351745,17247.876124
4,2021-11-05,28152.938540,34873.939080,59968.393160,143505.827198,108761.777883,18078.266972,46975.701291,84943.135732,100294.829339,...,49427.019462,47063.105078,86397.651814,34062.808374,26472.657621,55323.587424,321108.356663,264131.897754,27408.366650,17492.773824
5,2021-11-29,27919.530611,34687.673458,60773.779528,142621.815394,104901.698658,16669.447967,47219.595113,85236.833170,103490.352393,...,49749.405974,51984.322942,84384.021784,36846.592704,25721.026664,53328.350326,336697.743579,262257.538308,27176.085090,17382.219194
6,2021-11-30,28750.750484,35032.651375,60312.130021,143012.927861,107216.342323,17001.594758,46672.351191,85360.327648,102788.687368,...,48923.253693,51539.564130,84739.157977,35258.227509,25330.528060,53645.466661,335662.149461,264562.230652,27415.549191,17338.522537
7,2021-12-01,28858.095631,34995.888574,60241.470410,145626.792237,111192.733424,17665.577952,45757.683516,85665.326378,102943.419081,...,48834.428543,49341.153729,85450.806060,35320.479243,25623.131825,55136.121480,329167.718872,264623.119599,27466.446666,17286.577226
8,2021-12-02,28901.301911,34866.098057,59701.569734,145351.536595,109530.741544,17846.022961,46380.152350,85494.864447,99958.476851,...,48954.935640,48626.256108,84823.889868,34343.016848,25878.115140,55964.135511,329482.609718,261821.140588,27594.351745,17247.876124
9,2021-12-03,28152.938540,34873.939080,59968.393160,143505.827198,108761.777883,18078.266972,46975.701291,84943.135732,100294.829339,...,49427.019462,47063.105078,86397.651814,34062.808374,26472.657621,55323.587424,321108.356663,264131.897754,27408.366650,17492.773824


In [17]:
valid_data = pd.DataFrame()

start_date = '20211101'
end_date = '20211105'
for stock_code in stock_list["종목코드"]:
    stock = fdr.DataReader(stock_code, start = start_date, end = end_date)
    valid_data = pd.concat([valid_data,stock["Close"]], axis=1)
valid_data.columns = stock_list["종목명"]
valid_data = valid_data[stock_list.sort_values("종목코드")["종목명"]]
valid_data


종목명,메리츠화재,하이트진로,유한양행,CJ대한통운,두산,한국앤컴퍼니,삼천당제약,기아,SK하이닉스,영풍,...,네패스아크,두산퓨얼셀,솔루스첨단소재,알체라,고바이오랩,넥스틴,하이브,솔브레인,티와이홀딩스,엑세스바이오
2021-11-01 00:00:00,27850,35200,60000,146000,103000,16600,49750,84300,106500,694000,...,48300,51700,80500,36750,25700,52000,348500,261600,26600,17600
2021-11-02 00:00:00,29250,35050,61700,148500,107000,17350,48950,86000,107500,698000,...,50800,54100,81600,35850,25950,52900,348000,258600,27100,18100
2021-11-03 00:00:00,30250,34050,61500,145500,110000,17100,50300,85000,105500,685000,...,49450,54600,82700,34700,25450,51200,346500,253700,26750,18300
2021-11-04 00:00:00,29450,33800,61100,145000,125500,17750,50600,87000,106000,687000,...,48700,53800,81500,35500,25500,50600,356500,249600,26350,17550
2021-11-05 00:00:00,29550,33450,60600,144500,133500,17300,50400,88000,107000,684000,...,49800,53900,85100,35500,25100,50200,383500,247800,26050,17550


In [18]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(valid_data, sample_submission.iloc[:5, 1:])

3030.889167683397